# Case: Previsão do preço de imóveis no Rio de Janeiro utilizando regressão

## Contexto
O mercado imobiliário é um importante setor da economia brasileira, e a previsão do preço de imóveis é uma questão relevante tanto para compradores quanto para vendedores. Neste caso, o objetivo é desenvolver um modelo de regressão para prever o preço de imóveis na cidade do Rio de Janeiro, utilizando dados públicos disponíveis.

## Base de dados
O Inside Airbnb (http://insideairbnb.com/get-the-data.html) oferece dados sobre anúncios de acomodações no Airbnb, incluindo informações sobre localização, tipo de imóvel, número de quartos e preços. É possível utilizar esses dados para modelar o preço dos imóveis no Rio de Janeiro, filtrando as informações relevantes para o contexto local.

## Aplicação de Ciência de Dados

### 1. Coleta e limpeza de dados
Baixe o arquivo "listings.csv" referente à cidade do Rio de Janeiro no Inside Airbnb. Em seguida, realize a limpeza dos dados removendo entradas duplicadas, tratando valores ausentes e convertendo variáveis categóricas em numéricas.
 - bronze: dado coletado em formato legivel (.parquet)
 - silver: dado filtrado ou convertido em formato (string -> date)
 - gold: dado do imóvel (id, preço, quantidade de quartos)

### 2. Análise exploratória de dados (EDA)
A EDA é realizada para identificar padrões, tendências e correlações entre as variáveis. Por exemplo, verificar a distribuição dos preços dos imóveis, identificar quais bairros possuem os imóveis mais caros e analisar a relação entre o tamanho do imóvel e o preço.

### 3. Feature engineering
Crie novas variáveis relevantes, como a distância até o centro da cidade ou a presença de comércio e serviços nas proximidades. Essas variáveis podem ajudar a melhorar o desempenho do modelo de regressão.

### 4. Modelagem e algoritmos
Separe a base de dados em conjuntos de treinamento e teste. Treine diferentes modelos de regressão, como regressão linear, árvores de decisão e regressão de floresta aleatória (Random Forest). Utilize validação cruzada para avaliar o desempenho dos modelos e selecione aquele com a melhor performance.

### 5. Avaliação e interpretação
Avalie o modelo selecionado utilizando métricas de desempenho, como o erro médio quadrático (RMSE) e o coeficiente de determinação (R²). Interprete os resultados, identificando os fatores que mais influenciam o preço dos imóveis e fornecendo insights para os interessados no mercado imobiliário.

Dessa forma, é possível criar um modelo de regressão para prever o preço de imóveis no Rio de Janeiro utilizando dados disponíveis e técnicas de ciência de dados.


#### Carregando os Dados

In [1]:
!pip install geopy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.6 MB/s eta 0:00:00


In [2]:
import pandas as pd
import pyarrow.parquet as pq
import re
from geopy.distance import distance
import matplotlib.pyplot as plt
import math

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
mount_path = "/workspaces/prediction_house_price_from_airbnb/Data"

In [92]:
# read the Parquet file into a pyarrow Table

listings = pd.read_parquet(mount_path + "/bronze/listings.parquet")
#neighbourhoods = pd.read_parquet(mount_path + "/bronze/neighbourhoods.parquet")
#reviews = pd.read_parquet(mount_path + "/bronze/reviews.parquet")
#calendar = pd.read_parquet(mount_path + "/bronze/calendar.parquet")



In [6]:
def clean_price_column(df):
    """
    Limpa a coluna 'price' do DataFrame 'df', removendo o símbolo de dólar ($) e a vírgula (,)
    e convertendo o resultado em um valor float. A nova coluna 'price' é adicionada ao DataFrame 'listings'.

    Args:
        df (pandas.DataFrame): O DataFrame a ser limpo.

    Returns:
        pandas.DataFrame: Uma cópia do DataFrame original com a nova coluna 'price' adicionada,
        contendo os valores de preço limpos e convertidos em float.
    """
    listings = df.copy() # cria uma cópia do DataFrame original
    listings['price'] = listings['price'].str.replace('[$,]', '', regex=True).astype(float) # substitui o símbolo de dólar e vírgula e converte para float
    return listings

def extract_bathrooms(df):
    """
    Extrai o número de banheiros da coluna 'bathrooms_text' do DataFrame 'df' e o converte para float,
    salvando o resultado na nova coluna 'bathrooms'.

    Args:
        df (pandas.DataFrame): O DataFrame a ser processado.

    Returns:
        pandas.DataFrame: Uma cópia do DataFrame original com a nova coluna 'bathrooms' adicionada,
        contendo os valores de banheiros extraídos e convertidos em float.
    """
    listings = df.copy() # cria uma cópia do DataFrame original
    listings['bathrooms'] = listings['bathrooms_text'].str.extract(r'(\d+(?:\.\d+)?)?').astype(float) # extrai o número de banheiros e converte para float
    return listings




def calc_distance_cen(row):
    """
    Calcula a distância entre as coordenadas geográficas do imóvel representado pela linha 'row' e as coordenadas do centro do Rio de Janeiro.
    Retorna o valor da distância em quilômetros.

    Args:
        row (pandas.Series): Uma linha do DataFrame 'listings' contendo informações sobre um imóvel.

    Returns:
        float: A distância em quilômetros entre o imóvel e o centro do Rio de Janeiro.
    """
    # Coordenadas geográficas do centro do Rio de Janeiro
    center_lat = -22.908333
    center_lon = -43.196388

    # Coordenadas geográficas do imóvel
    lat = row['latitude']
    lon = row['longitude']

    # Conversão de graus para radianos
    lat_rad = math.radians(lat)
    lon_rad = math.radians(lon)
    center_lat_rad = math.radians(center_lat)
    center_lon_rad = math.radians(center_lon)

    # Cálculo da distância entre as coordenadas geográficas
    delta_lat = center_lat_rad - lat_rad
    delta_lon = center_lon_rad - lon_rad
    a = math.sin(delta_lat/2)**2 + math.cos(lat_rad) * math.cos(center_lat_rad) * math.sin(delta_lon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    distance = 6371 * c # raio médio da Terra em quilômetros

    return distance


def calc_distance_cor(row):
    """
    Calcula a distância entre as coordenadas geográficas do imóvel representado pela linha 'row' e as coordenadas do centro do Rio de Janeiro.
    Retorna o valor da distância em quilômetros.

    Args:
        row (pandas.Series): Uma linha do DataFrame 'listings' contendo informações sobre um imóvel.

    Returns:
        float: A distância em quilômetros entre o imóvel e o centro do Rio de Janeiro.
    """
    # Coordenadas geográficas do centro do Rio de Janeiro
    center_lat = -22.9524
    center_lon = -43.2114

    # Coordenadas geográficas do imóvel
    lat = row['latitude']
    lon = row['longitude']

    # Conversão de graus para radianos
    lat_rad = math.radians(lat)
    lon_rad = math.radians(lon)
    center_lat_rad = math.radians(center_lat)
    center_lon_rad = math.radians(center_lon)

    # Cálculo da distância entre as coordenadas geográficas
    delta_lat = center_lat_rad - lat_rad
    delta_lon = center_lon_rad - lon_rad
    a = math.sin(delta_lat/2)**2 + math.cos(lat_rad) * math.cos(center_lat_rad) * math.sin(delta_lon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    distance = 6371 * c # raio médio da Terra em quilômetros

    return distance


def calc_distance_gal(row):
    """
    Calcula a distância entre as coordenadas geográficas do imóvel representado pela linha 'row' e as coordenadas do centro do Rio de Janeiro.
    Retorna o valor da distância em quilômetros.

    Args:
        row (pandas.Series): Uma linha do DataFrame 'listings' contendo informações sobre um imóvel.

    Returns:
        float: A distância em quilômetros entre o imóvel e o centro do Rio de Janeiro.
    """
    # Coordenadas geográficas do centro do Rio de Janeiro
    center_lat = -22.8053
    center_lon = -43.2566

    # Coordenadas geográficas do imóvel
    lat = row['latitude']
    lon = row['longitude']

    # Conversão de graus para radianos
    lat_rad = math.radians(lat)
    lon_rad = math.radians(lon)
    center_lat_rad = math.radians(center_lat)
    center_lon_rad = math.radians(center_lon)

    # Cálculo da distância entre as coordenadas geográficas
    delta_lat = center_lat_rad - lat_rad
    delta_lon = center_lon_rad - lon_rad
    a = math.sin(delta_lat/2)**2 + math.cos(lat_rad) * math.cos(center_lat_rad) * math.sin(delta_lon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    distance = 6371 * c # raio médio da Terra em quilômetros

    return distance

def calc_distance_oli(row):
    """
    Calcula a distância entre as coordenadas geográficas do imóvel representado pela linha 'row' e as coordenadas do centro do Rio de Janeiro.
    Retorna o valor da distância em quilômetros.

    Args:
        row (pandas.Series): Uma linha do DataFrame 'listings' contendo informações sobre um imóvel.

    Returns:
        float: A distância em quilômetros entre o imóvel e o centro do Rio de Janeiro.
    """
    # Coordenadas geográficas do centro do Rio de Janeiro
    center_lat = -22.9774
    center_lon = -43.3940
    # Coordenadas geográficas do imóvel
    lat = row['latitude']
    lon = row['longitude']

    # Conversão de graus para radianos
    lat_rad = math.radians(lat)
    lon_rad = math.radians(lon)
    center_lat_rad = math.radians(center_lat)
    center_lon_rad = math.radians(center_lon)

    # Cálculo da distância entre as coordenadas geográficas
    delta_lat = center_lat_rad - lat_rad
    delta_lon = center_lon_rad - lon_rad
    a = math.sin(delta_lat/2)**2 + math.cos(lat_rad) * math.cos(center_lat_rad) * math.sin(delta_lon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    distance = 6371 * c # raio médio da Terra em quilômetros

    return distance

def calc_distance_cop(row):
    """
    Calcula a distância entre as coordenadas geográficas do imóvel representado pela linha 'row' e as coordenadas do centro do Rio de Janeiro.
    Retorna o valor da distância em quilômetros.

    Args:
        row (pandas.Series): Uma linha do DataFrame 'listings' contendo informações sobre um imóvel.

    Returns:
        float: A distância em quilômetros entre o imóvel e o centro do Rio de Janeiro.
    """
    # Coordenadas geográficas do centro do Rio de Janeiro
    center_lat = -22.9739
    center_lon = -43.1853

    # Coordenadas geográficas do imóvel
    lat = row['latitude']
    lon = row['longitude']

    # Conversão de graus para radianos
    lat_rad = math.radians(lat)
    lon_rad = math.radians(lon)
    center_lat_rad = math.radians(center_lat)
    center_lon_rad = math.radians(center_lon)

    # Cálculo da distância entre as coordenadas geográficas
    delta_lat = center_lat_rad - lat_rad
    delta_lon = center_lon_rad - lon_rad
    a = math.sin(delta_lat/2)**2 + math.cos(lat_rad) * math.cos(center_lat_rad) * math.sin(delta_lon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    distance = 6371 * c # raio médio da Terra em quilômetros

    return distance

In [32]:
listings=clean_price_column(listings)
listings=extract_bathrooms(listings)
listings['distance_centro'] = listings.apply(calc_distance_cen, axis=1)
listings['distance_corcovado'] = listings.apply(calc_distance_cor, axis=1)
listings['distance_galeao'] = listings.apply(calc_distance_gal, axis=1)
listings['distance_olipico'] = listings.apply(calc_distance_oli, axis=1)
listings['distance_copacabana'] = listings.apply(calc_distance_cop, axis=1)

In [ ]:
# Remover as aspas das strings de amenities
listings['amenities'] = listings['amenities'].str.replace('"', '')
# Obter todos os amenities únicos em uma lista
unique_amenities = list(set([amenity for amenities in listings['amenities'] for amenity in amenities]))
# Criar colunas separadas para cada amenity em uma lista
for amenity in unique_amenities:
    listings[amenity] = listings['amenities'].apply(lambda x: 1 if amenity in x else 0)
    
listings[['last_scraped', 'host_since','calendar_last_scraped','first_review','last_review']] =( 
    listings[['last_scraped', 'host_since','calendar_last_scraped','first_review','last_review']].apply(pd.to_datetime))
# definir uma data fixa
data_fixa = pd.to_datetime('2023-05-01')

# calcular a diferença em anos entre cada data e a data fixa
listings['years'] = (data_fixa - listings['host_since']).dt.days / 365.25

In [ ]:
listings.to_parquet('/workspaces/prediction_house_price_from_airbnb/Data/silver/listings_v01.parquet')

In [106]:
listings.head(2)

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,E,N,r,S,*,t,),a,;,],I,v,x,2,8,U,4,,",",-,p,F,1,n,B,[,+,K,5,\,c,.,|,%,H,6,',j,m,D,f,h,:,X,T,w,!,o,V,O,e,P,J,i,d,Z,3,7,z,W,A,/,L,G,9,$,k,Y,b,l,g,0,y,Q,C,u,s,q,`,&,M,R,(,since,years
0,783493769216852616,https://www.airbnb.com/rooms/783493769216852616,20221229002515,2022-12-29,city scrape,"Leme, Brasil",Apartamento aconchegante próximo à praia do Le...,None,https://a0.muscache.com/pictures/fe1f4b1b-a300...,491704706,https://www.airbnb.com/users/show/491704706,Felipe,2022-12-16,None,None,within a few hours,100%,67%,f,https://a0.muscache.com/defaults/user_pic-50x5...,https://a0.muscache.com/defaults/user_pic-225x...,Leme,2.0,2.0,"['email', 'phone']",f,f,None,Leme,NaN,-22.96421,-43.171600,Entire rental unit,Entire home/apt,2,NaN,1 bath,1.0,1.0,"[Wifi, Dedicated workspace, Kitchen, Fire exti...",$720.00,1,365,1,1,365,365,1.0,365.0,NaN,t,27,57,87,362,2022-12-29,0,0,0,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,2,2,0,0,NaN,0,0,1,0,0,1,0,1,0,1,0,0,1,0,0,0,0,1,1,0,1,1,0,1,1,1,0,1,0,0,1,0,0,0,0,0,0,0,1,1,1,1,0,0,1,1,0,1,1,0,1,0,0,1,1,0,0,0,0,1,1,0,1,0,0,0,1,0,0,1,1,0,1,0,0,1,1,0,0,0,0,0,0,0.372348,0.372348
1,703973293620197060,https://www.airbnb.com/rooms/703973293620197060,20221229002515,2022-12-29,city scrape,Suíte com entrada independente em casarão 1,"Casa ampla, em excelente localização, situada ...",None,https://a0.muscache.com/pictures/miso/Hosting-...,20362236,https://www.airbnb.com/users/show/20362236,Júlio Cesar,2014-08-21,"Rio de Janeiro, Brazil",None,within an hour,70%,100%,f,https://a0.muscache.com/im/users/20362236/prof...,https://a0.muscache.com/im/users/20362236/prof...,Botafogo,4.0,5.0,"['email', 'phone']",t,t,None,Botafogo,NaN,-22.95792,-43.182226,Private room in bed and breakfast,Private room,2,NaN,1 private bath,1.0,1.0,"[Wifi, Lock on bedroom door, TV, Coffee maker,...",$599.00,4,365,4,4,365,365,4.0,365.0,NaN,t,28,58,88,363,2022-12-29,1,1,0,2022-09-06,2022-09-06,5.0,5.0,5.0,5.0,5.0,4.0,5.0,NaN,t,4,0,4,0,0.26,0,0,1,0,0,1,0,1,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,1,1,0,1,1,0,1,1,0,1,1,0,0,0,0,1,0,0,1,0,0,0,1,0,1,1,1,0,1,0,1,0,1,0,0,0,0,0,0,8.692676,8.692676


In [99]:
listings.dtypes

id               int64
listing_url     object
scrape_id        int64
last_scraped    object
source          object
                 ...  
`                int64
&                int64
M                int64
R                int64
(                int64
Length: 158, dtype: object